# Data Description

For our project, we collected wikipedia articles corresponding to five major social science diciplines: Economics, Political Science, Anthropology, Sociology and Psychology. To do this we used the tool [petscan](https://petscan.wmflabs.org/) which enables one to find all the sub-categories/articles corresponding to a given page. Petscan can be access both through a web-interface or programmatically through for example Python. We did it programmatically by querying the main page corresponding to each discipline.

**Insert GIF about HERE**

In [31]:
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)
import pandas as pd
from itables import show

df = pd.read_pickle('/home/matiasp/University/m2/socialgraphs2021-Matias/project/wiki_df.pickle')
sub = df[['name', 'parent']].sample(500).reset_index(drop = True)
sub = sub.rename(columns={'name':'Article Name', 'parent':'discipline'})
sub

<IPython.core.display.Javascript object>

Article Name,discipline
